# Warehouse Simulation Based on SimPy
This notebook provides the code for simulating warehouse activity using several algorithm to pick the order

## Flow
The flow of this simulation code is as follow:
1. prepare required module
1. define combination of rules for order picking
1. prepare SimPy environment
1. run simulation
1. generate report

### 1. Prepare required module

In [6]:
import simpy
import pandas as pd
import random
from datetime import timedelta
from modules import general_function as gf
from modules import pooling_triggers
from modules import batchings
from modules import routings

### 2. Define rules combination

| Option    | Trigger   | Batching  | Routing       | Cart  |
| ---       | ---       | ---       | ---           | ---   |
| 1         | FTWB      | FCFS      | S-Shape       | 50    |
| 2         | VTWB      | Seed      | Largest Gap   | 100   |
| 3         | Max Picker |          |               | 200   |
| 4         | Max Cart
| 5         | Urgent First + Max Picker
| 6         | Urgent First + Max Cart


In [7]:
trigger_opt = 1
batching_opt = 1
routing_opt = 1
picker_num = 1
cart_opt = 1

### 3. Prepare SimPy environment

In [8]:
# Reading order file
fname = gf.reading_file()

# Processing file
# Making Index
a = len(fname) + 1
orderFile = list(range(1, a)) + ['Avg']
# Counting total order
# totalOrder = gf.count_total_order(fname)
totalOrder = list()
# Counting total item picked
# totalItemPicked = gf.count_total_item(fname)
totalItemPicked = list()

# Prepare report container
totalCompletionTime = list()
totalTurnOverTime = list()
averagePickerUtility = list()
averageCartUtility = list()
lateDelivery = list()

In [9]:
# Assign cart capacity
cart_capacity = 0
if cart_opt == 1:
    cart_capacity = 50
elif cart_opt == 2:
    cart_capacity = 100
elif cart_opt == 3:
    cart_capacity = 200

### 4. Run Simulation

In [10]:
for fn in fname:
    # env = simpy.Environment()
    class Object(object):
        pass
    env = Object()
    env.now = 0
    # Setting delta for FTWB
    delta = timedelta(minutes=12).seconds

    # Determine batching method
    batching = batchings.batchings(batching_opt, cart_capacity)
    # Determine routing method
    routing = routings.routings(batching_opt)
    # Prepare pooling and determine trigger method
    trigger = pooling_triggers.pooling_triggers(trigger_opt, env, picker_num, batching, routing, cart_capacity, delta)

    # Define simulation time
    limit = timedelta(hours=8).seconds

    # Run simulation
    # env.run(until=limit)
    
    trigger.run(fn, limit)

    # Prepare env processing
    # env.process(trigger.run_simpy(fn, limit))

    # Listing total order
    totalOrder.append(trigger.currentRow - 1)
    # Listing total total time
    totalItemPicked.append(trigger.processed_item)
    # Listing total completion time
    totalCompletionTime.append(trigger.completionTime)
    # Listing total Turonver time
    totalTurnOverTime.append(trigger.turnOverTime)
    # Counting and listing average picker utility
    avePickerUtility = round(trigger.completionTime/(timedelta(hours=8)*picker_num), 2)
    averagePickerUtility.append(avePickerUtility)
    # Counting & listing average cart utility
    aveCartUtility = round(trigger.cartUtility/trigger.num_triggered, 2)
    averageCartUtility.append(aveCartUtility)
    # Listing total on time delivery
    lateDelivery.append(trigger.lateCount)

In [11]:
# Counting average of total order
totalOrder = gf.count_average(totalOrder)
# Counting average of total item picked
totalItemPicked = gf.count_average(totalItemPicked)
# Counting average of total completion time
totalCompletionTime = gf.average_tct(totalCompletionTime)
# Counting average of turnover time
totalTurnOverTime = gf.average_tct(totalTurnOverTime)
# Counting average of average picker utility
averagePickerUtility = gf.count_average(averagePickerUtility)
# Counting average of average cart utility
averageCartUtility = gf.count_average(averageCartUtility)
# Counting average of on time delivery
lateDelivery = gf.count_average(lateDelivery)

### 5. Generate Report

In [12]:
# Put result on file
result = pd.DataFrame({
    'OrderFile': orderFile,
    'TriggerMethod': [trigger_opt] * 11,
    'BatchingMethod': [batching_opt] * 11,
    'RoutingPolicy': [routing_opt] * 11,
    'NumOfPickers': [picker_num] * 11,
    'CartCapacity': [cart_capacity] * 11,
    'TotalOrder': totalOrder,
    'CompletionTime': totalCompletionTime,
    'TurnOverTime': totalTurnOverTime,
    'TotalItemPicked': totalItemPicked,
    'AvgPickerUtil': averagePickerUtility,
    'AvgCartUtil': averageCartUtility,
    'NumOfLate': lateDelivery
    }, columns = ['OrderFile','TriggerMethod','BatchingMethod','RoutingPolicy','NumOfPickers','CartCapacity','TotalOrder','CompletionTime','TurnOverTime','TotalItemPicked','AvgPickerUtil','AvgCartUtil','NumOfLate'])
filename = str(trigger_opt) + str(batching_opt) + str(routing_opt) + str(picker_num) + str(cart_opt) + '.csv'
result.to_csv('result/' + filename, index = False)
print(filename)

11111.csv
